In [ ]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        images.append((filename, img))
    return images

def compute_hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block, visualize=True)
    hog_image = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd, hog_image

def find_eye_template(template, image, threshold=0.7):
    # Вычисляем HOG для шаблона и изображения
    _, hog_template = compute_hog(template)
    _, hog_image = compute_hog(image)
    
    # Нормализация
    hog_template = cv2.normalize(hog_template, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    hog_image = cv2.normalize(hog_image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    
    res = cv2.matchTemplate(hog_image, hog_template, cv2.TM_CCOEFF_NORMED)  
    
    loc = np.where(res >= threshold)

    # Получаем все точки, превышающие порог
    y_coords, x_coords = np.where(res >= threshold)
    
    # Фильтруем точки, которые слишком близко друг к другу
    filtered_points = []

    min_dist = 10

    for i in range(len(x_coords)):
        keep = True
        x, y = x_coords[i], y_coords[i]
        
        # Проверяем расстояние до всех уже добавленных точек
        for (fx, fy) in filtered_points:
            if abs(x - fx) < min_dist or abs(y - fy) < min_dist:
                keep = False
                continue
                
        if keep:
            filtered_points.append((x, y))

    return loc
    
    # Разделяем обратно на координаты x и y для возврата
    if filtered_points:
        return tuple(zip(*filtered_points))  # Возвращаем в том же формате, что и np.where
    else:
        return (np.array([], dtype=np.int64), np.array([], dtype=np.int64))
    
    

def process_images(template_path, images_folder, output_folder):
    
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    
    os.makedirs(output_folder, exist_ok=True)    
    
    images = load_images_from_folder(images_folder)
    
    for filename, img in images:
        # Находим шаблон на изображении
        loc = find_eye_template(template, img)
        
        # Будем рисовать прямоугольники вокруг найденных глаз
        img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        w, h = template.shape[::-1]
        
        for pt in zip(*loc[::-1]):
            cv2.rectangle(img_color, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
        
        # Результаты находятся в папке results
        output_path = os.path.join(output_folder, f"result_{filename}")
        cv2.imwrite(output_path, img_color)


template_path = "hog/eye.png"  # Путь к шаблону глаза
images_folder = "hog/images"   # Папка с изображениями
output_folder = "hog/results"  # Папка для результатов

process_images(template_path, images_folder, output_folder)
print("Обработка завершена. Результаты сохранены в", output_folder)

Обработка завершена. Результаты сохранены в hog/results
